In [1]:
import os
import pandas as pd
from pnp.physical import ftm

folds = ["test", "train", "val"]
fold_dfs = {}

for fold in folds:
    csv_name = fold + "_param_v2.csv"
    csv_path = os.path.join("..", "data", csv_name)
    fold_df = pd.read_csv(csv_path)
    fold_df = fold_df.rename(columns={"Unnamed: 0": "ID"})
    fold_dfs[fold] = fold_df
    
full_df = pd.concat(fold_dfs.values()).sort_values(
    by="ID", ignore_index=False)
assert len(set(full_df["ID"])) == len(full_df)

In [4]:
theta_keys = ["omega", "tau", "p", "D", "alpha"]
row = full_df.iloc[0]
theta = {key: row[key] for key in theta_keys}
ftm.rectangular_drum(**theta, **ftm.constants)

array([0.00000000e+00, 4.46210975e-05, 8.89881563e-05, ...,
       1.33345855e-14, 1.29237385e-14, 1.25466374e-14])